![Logo de AA1](logo_AA1_texto_small.png) 
# Sesión 20 - Naïve Bayes

Habitualmente tenemos nuestros ejemplos definidos como pares $(x,y)$, donde $x$ es un vector que contiene los valores que presenta un ejemplo para cada uno de sus atributos ($x=(x_1, \dots, x_d)$) e $y$ se corresponde con la clase de ese ejemplo. Para responder a la pregunta ¿cuál es la probabilidad de que un ejemplo descrito con los valores $x$ sea de la clase $y$? podemos hacer uso de modelos probabilísticos recurriendo al Teorema de Bayes:

$$
P(A|B) = \frac{P(B|A)P(A)}{P(B)}
$$

Para poder aplicar este teorema en el problema que nos ocupa debemos hacer una asunción fuerte: *los atributos de cualquier ejemplo son independientes entre sí una vez que sabemos a qué clase pertenece el ejemplo*.

Realizando esta asunción podemos formular el clasificador **Naïve Bayes**:

$$
P(y|x_1, \dots, x_d) = \frac{P(x_1, \dots, x_d|y)P(y)}{P(x_1, \dots, x_d)}
$$

donde:
- $P(y)$ es la *probabilidad a priori* de que un ejemplo sea de la clase $y$
- $P(x_1, \dots, x_d)$ es la probabilidad de que se dé un ejemplo con los valores $(x_1, \dots, x_d)$ en los atributos. Esta probabilidad es conocida como la *probabilidad marginal*
- $P(x_1, \dots, x_d|y)$ es la probabilidad de que un ejemplo de la clase $y$ tenga los valores $(x_1, \dots, x_d)$ en los atributos. Esta probabilidad se conoce como *verosimilitud* (*likelihood*)
- $P(y|x_1, \dots, x_d)$ es la probabilidad de que dado un ejemplo con los valores $(x_1, \dots, x_d)$ en los atributos sea de la clase $y$. Esta probabilidad es conocida como *probabilidad a posteriori*

También debemos hacer otra asunción sobre la distribución que sigue la verosimilitud de cada atributo, $P(x_i|y)$. Se suele trabajar con 3 distribuciones diferentes:
1. Normal. Asumimos que los valores de los atributos siguen una campana de Gauss
2. Multinomial. Asumimos que los atributos pueden tomar un subconjunto discreto de valores
3. Bernoulli. Asumimos que los atributos pueden tomar dos valores discretos

`Sckit-learn` tiene implementado un algoritmo para cada una de estas tres asunciones. Nos centraremos en la primera de ellas puesto que es la más utilizada y veremos información de las otras dos aunque no trabajaremos con ellas.

## 20.1 `GaussianNB`
Como habitualmente trabajamos con valores continuos en los atributos es muy habitual realizar la asunción de que los atributos siguen una distribución normal y eso hace que `GaussianNB` sea el clasificador Naïve Bayes más utilizado.

Vamos a cargar un conjunto de datos:

In [ ]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn import metrics

print('\n##########################################')
print('### cargar el conjunto y separar X e y')
print('##########################################')

# se llama a la función read_csv
# no tiene missing y las columnas están separadas por ','
# tampoco cabecera, así que hay que dar nombre a las columnas (como en el names no vienen indicados creamos nombres)
cabecera = ['atr'+str(x) for x in range(1,35)]
cabecera.append('clase')
df = pd.read_csv('ionosphere.data', names=cabecera)
filas, columnas = df.shape

# se crea el objeto LabelEncoder para transformar la clase
class_enc = LabelEncoder() 

# se transforma la clase
df['clase'] = class_enc.fit_transform(df['clase'])
print("Clases:", class_enc.classes_)

# separamos los atributos y los almacenamos en X
X = df.drop(['clase'], axis=1)
display(X)

# separamos la clase y la almacenamos en Y
y = df['clase']
display(y)

Separamos unos ejemplos como conjunto de test:

In [ ]:
print('\n##########################################')
print('### Hold-out 70-30')
print('##########################################')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234, stratify=y)

Ahora creamos un clasificador `GaussianNB` y obtenemos su accuracy en el conjunto de test:

In [ ]:
# Naive Bayes
sys_nb = GaussianNB()
sys_nb.fit(X_train, y_train) 
y_pred = sys_nb.predict(X_test)
print("Accuracy: %.4f" % metrics.accuracy_score(y_test, y_pred))

`GaussianNB` es un algoritmo con muy pocos hiperparámetros: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB 

Únicamente el hiperparámetro `var_smoothing` puede influir en el rendimiento del sistema. En caso de que influya, las variaciones en el rendimiento no suelen ser muy grandes como ocurre con los hiperparámetros de otros sistemas.

In [ ]:
print('\n##########################################')
print('### se busca el mejor valor para var_smoothing')
print('##########################################')

# se definen los valores de var_smoothing que se quieren probar
valores_de_smoothing = [1e-1, 1e-3, 1e-6, 1e-9, 1e-12, 1e-20]

# se crea un generador de folds estratificados partiendo el conjunto en 5 trozos
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# creamos la grid search 
gs_nb = GridSearchCV(sys_nb, param_grid={'var_smoothing':valores_de_smoothing}, scoring='accuracy', cv=folds, verbose=1, n_jobs=-1)

# ejecutamos la búsqueda
gs_nb_trained = gs_nb.fit(X_train, y_train)

# resultados
print("Accuracy para cada valor:", gs_nb_trained.cv_results_['mean_test_score'])
print("Mejor combinación de hiperparámetros:", gs_nb_trained.best_params_)
print("Mejor rendimiento obtenido en GridSearch: %.4f" % gs_nb_trained.best_score_)
y_pred = gs_nb_trained.predict(X_test)
print("Accuracy en el conjunto de test: %.4f" % metrics.accuracy_score(y_test, y_pred))

Vemos que los resultados obtenidos para cada valor de `var_smoothing` son similares y para valores muy extremos de dicho hiperparámetro no observamos rendimientos escandalosamente malos.

Además, también podemos apreciar que el rendimiento sobre el conjunto de test es peor que el que teníamos antes, aunque ahora hayamos seleccionado el mejor valor para el hiperparámetro. Una de las razones para que esto haya sucedido es que puede ser que el conjunto de datos separado para test tenga unas características particulares que provoquen esta situación. Por eso, siempre será mejor evaluar utilizando algún método que garantice que todos los ejemplos aparezcan en el conjunto de test, como sucede cuando utilizamos una validación cruzada o un leave-one-out (aunque a veces no podremos utilizarlos por su coste computacional).

## 20.2 `MultinomialNB` y `BernoulliNB`

Cuando los atributos son discretos pueden seguir una distribución multinomial o de Bernoulli. Será de Bornoulli si los atributos pueden tomar únicamente dos valores y multinomial si pueden tomar más valores.

Estas son situaciones que suelen darse a menudo cuando se trabaja en clasificación de textos, algo que está fuera de los objetivos de esta asignatura y, por tanto, os dejamos cierta información que **NO formará parte de la evaluación de esta asignatura**.

Dentro de `Sckit-learn` los algoritmos para trabajar bajo estas asunciones son:
- `BernoulliNB` https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB
- `MultinomialNB` https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB 

El campo del análisis de textos es muy amplio y en este grado hay una asignatura exclusiva enfocada en su estudio. Aquí os dejamos unos enlaces por si tenéis curiosidad:
- https://en.wikipedia.org/wiki/Text_mining
- https://en.wikipedia.org/wiki/Document_classification
- https://en.wikipedia.org/wiki/Sentiment_analysis 

## 20.3 Calibrado de las probabilidades

Las probabilidades que genera `GaussianNB` al utilizar `predict_proba()` no son buenas y no pueden ser utilizadas como tales:

In [ ]:
# Naive Bayes
sys_nb = GaussianNB()
sys_nb.fit(X_train, y_train) 
y_pred = sys_nb.predict(X_test)
print("Accuracy: %.4f" % metrics.accuracy_score(y_test, y_pred))

# probabilidades
cuantos = 10
print("Probabilidades de los %d primeros ejemplos" % cuantos)
print(sys_nb.predict_proba(X_test.iloc[0:cuantos]))
print("Orden de las clases:", sys_nb.classes_)

Como vemos, las probabilidades obtenidas tienden a ser valores extremos muy cercanos a 0 o a 1. 

Estas probabilidades sí son útiles para establecer un ranking, es decir, ordenan bastante bien a los ejemplos en cuanto a su compatibilidad con cada clase. Sin embargo, no son útiles si se quiere ofrecer una predicción acompañada de una probabilidad: "*este ejemplo tiene una probabilidad del 78.3% de ser de la clase 1*". 

Si queremos obtener probabilidades que tengan sentido debemos utilizar la clase `CalibratedClassifierCV`, que utilizando una validación cruzada de k folds realiza un calibrado de las probabilidades para que se ajusten lo más posible a la realidad:

In [ ]:
# Naive Bayes
sys_nb = GaussianNB()

# se crea una instancia de CalibratedClassifierCV que utiliza el generador de fold creado anteriormente
sys_nb_calib = CalibratedClassifierCV(sys_nb, method='sigmoid', cv=folds)

# se entrena y se evalúa
sys_nb_calib.fit(X_train, y_train) 
y_pred = sys_nb_calib.predict(X_test)
print("Accuracy: %.4f" % metrics.accuracy_score(y_test, y_pred))

# probabilidades
cuantos = 10
print("Probabilidades de los %d primeros ejemplos" % cuantos)
print(sys_nb_calib.predict_proba(X_test.iloc[0:cuantos]))
print("Orden de las clases:", sys_nb_calib.classes_)

Vemos que las probabilidades obtenidas después de la calibración son más realistas. En este caso la accuracy coincide antes y después de calibrar, pero no tiene por qué ser así y, en ocasiones, el rendimiento puede cambiar considerablemente.

En el código anterior hemos creado una instancia de `CalibratedClassifierCV` que utiliza el *Platt scaling* (`method='sigmoid'`) para la calibración de las probabilidades. Si recordáis, este método es el que se utiliza para el cálculo de las probabilidades con las SVM.

En https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html?highlight=calibratedclassifiercv#sklearn.calibration.CalibratedClassifierCV podéis obtener más información.

`CalibratedClassifierCV` no es exclusivo para Naïve Bayes, puede utilizarse con otros algoritmos.

## Ejercicios

1. Carga el fichero **heart_failure_clinical_records_dataset.csv** (es un archivo de texto). 
2. Separar el conjunto en 70% para entrenar y 30% para test (estratificado)
3. Obtén la accuracy del `GaussianNB` con hiperparámetros por defecto.
4. Haz una búsqueda de `var_smoothing` con `GridSearchCV()` utilizando los ejemplos del conjunto de entrenamiento y evalúa el mejor modelo con el conjunto de test.
5. Realiza un calibrado de probabilidades y muestra las probabilidades obtenidas para los 5 primeros ejemplos

Estos ejercicios no es necesario entregarlos.